In [1]:
from googleplaces import GooglePlaces, types, lang
from googlemaps import Client as GoogleMaps
import pickle
import time

# pip install python-google-places, pickle

In [2]:
def Read_API_KEY_From_File(file_Path):
    list_Key = []
    with open(file_Path) as file_text:
        for key in file_text:
            list_Key.append(key.strip())
    return list_Key

In [3]:
api_key_File_Path = r'D:\Product\API_KEY\API_KEY.txt'

list_Key = Read_API_KEY_From_File(api_key_File_Path)

PLACE_API_KEY = list_Key[0]
ROADS_API_KEY = list_Key[1]

file_Path_To_Store_Data = 'Delhi_Proper_Places.pkl'

In [4]:
google_places = GooglePlaces(PLACE_API_KEY)

In [ ]:
latitude = 28.520516
longitude = 77.145236

location_To_Search = {'lat' : latitude, 'lng' : longitude}

time_To_Sleep = 10 # in secs
number_Of_Request_Required_For_Time_Sleep = 10

In [ ]:
# Below is boumdary range of Delhi state in box pattern
lat_Lower_Range = 28.404374
lat_Upper_Range = 28.88346
long_Lower_Range = 76.239726
long_Upper_Range = 77.343423

# $$dE0765017$$eE0772054$$fN0285252$$gN0282445

# Max Radius to search
max_Radius_Of_Search = 50000

# Searched places in Delhi
list_Places_Query_Result = []
list_Place_Id_In_Query_Result = []

# All types of places

all_Types_To_Search = [types.TYPE_TRAIN_STATION, types.TYPE_TAXI_STAND, types.TYPE_TRANSIT_STATION, 
                   types.TYPE_AIRPORT, types.TYPE_SHOPPING_MALL, types.TYPE_AMUSEMENT_PARK, types.TYPE_AQUARIUM, 
                   types.TYPE_BEAUTY_SALON, types.TYPE_CAFE, types.TYPE_BOWLING_ALLEY,  types.TYPE_GAS_STATION,  
                   types.TYPE_GROCERY_OR_SUPERMARKET, types.TYPE_GYM, types.TYPE_LIBRARY,  types.TYPE_HOSPITAL,  
                   types.TYPE_MOVIE_THEATER, types.TYPE_MUSEUM, types.TYPE_NIGHT_CLUB, types.TYPE_PARK, types.TYPE_PARKING, 
                   types.TYPE_RESTAURANT, types.TYPE_SPA, types.TYPE_ZOO, types.TYPE_BAR]

types_To_Search =  [ types.TYPE_HOSPITAL]


file_Path_To_Store_Data = 'Places.pkl'

In [ ]:
def Create_Location_Dictionary(latitude, longitude):
    return {'lat' : latitude, 'lng' : longitude}    

In [ ]:
def Get_Query_Result(location_To_Search, max_Radius, types_To_Search):
    
    return google_places.nearby_search(lat_lng=location_To_Search, radius=max_Radius, types = types_To_Search)

In [ ]:
def Is_Latitude_Longitude_In_Range(location):
    
    if  lat_Lower_Range <= location['lat'] and location['lat'] <= lat_Upper_Range:
        if long_Lower_Range <= location['lng'] and location['lng'] <= long_Upper_Range:
            return True
        else:
            return False
    else:
        return False

In [ ]:

def Find_All_Places(location):
    global list_Places_Query_Result
    global list_Place_Id_In_Query_Result
    global total_calls
    global types_To_Search
    global max_Radius_Of_Search
    
    query_Result = Get_Query_Result(location, max_Radius_Of_Search, types_To_Search)
    
    total_calls = total_calls + 1
    
    if (total_calls > 0 and total_calls % 10 == 0 ):
        print("Total Calls " + str(total_calls))
    
    if (total_calls > 0 and total_calls % number_Of_Request_Required_For_Time_Sleep == 0 ):
        time.sleep(time_To_Sleep)
    
    if(len(list_Places_Query_Result) > 0 and len(list_Places_Query_Result) % 100 == 0):
        print(" ##########################################   Total places found: " + str(len(list_Places_Query_Result)) )
        time.sleep(30)

    
    for place in query_Result.places:
        if Is_Latitude_Longitude_In_Range(place.geo_location):
            if place.place_id not in list_Place_Id_In_Query_Result:
                list_Places_Query_Result.append(place)
                list_Place_Id_In_Query_Result.append(place.place_id)
                Find_All_Places(place.geo_location)
        

In [ ]:
# Searched places in Delhi
# list_Places_Query_Result = []
# list_Place_Id_In_Query_Result = []

value_To_Increment = 0.2
count = 0


for type_Value in all_Types_To_Search:
    temp_lat = lat_Lower_Range
    temp_lng = long_Lower_Range

    count = count + 1
    types_To_Search = [type_Value]
    print (" @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  Type " + type_Value + "  " + str(count) + '/' + str(len(all_Types_To_Search)))
    
    if (count > 0):
        while temp_lat <= lat_Upper_Range:
            while temp_lng <=long_Upper_Range:
                location_To_Search = {'lat' : temp_lat, 'lng' : temp_lng}        
                Find_All_Places(location_To_Search)

                temp_lng = temp_lng + value_To_Increment

            temp_lat = temp_lat + value_To_Increment

        time.sleep(30)

In [ ]:
with open(file_Path_To_Store_Data, 'wb') as f:
    pickle.dump(list_Places_Query_Result, f)

In [17]:
import pickle

list_Places_Delhi = []
file_Path_To_Store_Data = 'Delhi_Proper_Places.pkl'

with open(file_Path_To_Store_Data, 'rb') as f:
    list_Places_Delhi = pickle.load(f)
    
len(list_Places_Delhi)

550

In [31]:
list_Place = []

for index in range (0, len(list_Places_Delhi), 100):
    list_Place = []
    
    for index_new in range (index, (index + 100),1 ):
        location_To_Search = list_Places_Delhi[index].geo_location
        list_Place.append(location_To_Search)
        
        result = gmaps.nearest_roads(list_Place)
        print(result)

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='geopy/1.16.0')

In [ ]:
delhi_places_Count = 0
count = 0
list_Proper_Places_Of_Delhi = []
list_Places_NOT_In_Delhi =[]

for places in list_Places_Delhi:
    list_Location = []
    list_Location.append(places.geo_location['lat'])
    list_Location.append(places.geo_location['lng'])
    location = geolocator.reverse(query=list_Location, timeout=10)
    
    if ( ('delhi' in str(location).lower()) and 
         ('haryana' not in str(location).lower()) and
         ('uttar pradesh' not in str(location).lower()) and 
         ('rajasthan' not in str(location).lower()) ):
        delhi_places_Count = delhi_places_Count + 1
        list_Proper_Places_Of_Delhi.append(places)
    
    else:
        list_Places_NOT_In_Delhi.append(places)
    
    count = count + 1
    
    if count % 50 == 0:
        print ("Proper Places of Delhi " + str(len(list_Proper_Places_Of_Delhi)))
        print ("Total places filtered " + str(count) + "/" + str(len(list_Places_Delhi)))
        time.sleep(5)

In [ ]:
print(len(list_Proper_Places_Of_Delhi))

In [ ]:
with open(file_Path_To_Store_Data, 'wb') as f:
    pickle.dump(list_Proper_Places_Of_Delhi, f)

In [ ]:
for places in list_Proper_Places_Of_Delhi:
    if 'bus' in (places.types[0]):
        print(places.types[0])

In [51]:
latitude = 28.520516
longitude = 77.145236

location_To_Search = {'lat' : latitude, 'lng' : longitude}

query_result = google_places.nearby_search(lat_lng=location_To_Search, radius=500, types = types.TYPE_ROUTE)

In [53]:
# for place in query_result.places:
#     print(place.get_details)

In [5]:

gmaps = GoogleMaps(ROADS_API_KEY)

In [15]:
location_To_Search = {'lat' : 28.612900001, 'lng' : 77.22950000001}

result = gmaps.nearest_roads(location_To_Search )

result

[]

In [ ]:
result = gmaps.nearest_roads(location_To_Search)

In [ ]:
for place in result:
    print(place)

In [34]:
places = []
placeId = 'ChIJqUkIFYEeDTkRSogx7POWbZU'

places.append(placeId)

place_Object = google_places.get_place(placeId)
print((place_Object))
print(place_Object.name)
print(place_Object.geo_location)


<Place name="Babu Jagjivan Ram Road", lat=28.5204357, lng=77.1443771>
Babu Jagjivan Ram Road
{'lat': Decimal('28.5204357'), 'lng': Decimal('77.1443771')}


In [6]:
#Creating famous tourist places coordinates

list_Known_Places = []
list_Known_Places.append((28.6129, 77.2295, 'Delhi Gate'))
# list_Known_Places.append((28.6127, 77.2773, 'Akshardham'))
list_Known_Places.append((28.6365, 77.1831, 'New Rajinder Nagar Post Office'))
list_Known_Places.append((28.5244, 77.1855, 'Qutub Minar'))
list_Known_Places.append((28.5681, 77.2058, 'Safdarjung Hospital'))
list_Known_Places.append((28.5997, 77.0990, 'Indira Park')) 
list_Known_Places.append((28.6390, 77.0745, 'PVR Vikaspuri')) 

In [115]:
def Get_Nearest_Road_Details(location_To_Search):
    result = gmaps.nearest_roads(location_To_Search)
    
    if (len(result) > 0):
        placeId = str(result[0]['placeId'])
        place_Object = google_places.get_place(placeId)
        return (place_Object)
    else:
        return False

In [117]:
for known_place in list_Known_Places:
    location_To_Search = {'lat' : known_place[0], 'lng' : known_place[1]}
    place = Get_Nearest_Road_Details(location_To_Search)
    
    print ()
    print ('Place Searched by   - ' + str(known_place[2]))
    print ('                          Location: ' + str(location_To_Search))
    
    if place != False:
        print ('Nearest Place found - ' + str(place.name))
        print ('                          Location: ' + str(place.geo_location))
    else:
        print('XXXXXXXXXXXXXXXX - No nearby places found')
    print ()


Place Searched by   - Delhi Gate
                          Location: {'lat': 28.6129, 'lng': 77.2295}
XXXXXXXXXXXXXXXX - No nearby places found


Place Searched by   - New Rajinder Nagar Post Office
                          Location: {'lat': 28.6365, 'lng': 77.1831}
Nearest Place found - Raman Lamba Marg
                          Location: {'lat': Decimal('28.6363491'), 'lng': Decimal('77.1831541')}


Place Searched by   - Qutub Minar
                          Location: {'lat': 28.5244, 'lng': 77.1855}
XXXXXXXXXXXXXXXX - No nearby places found


Place Searched by   - Safdarjung Hospital
                          Location: {'lat': 28.5681, 'lng': 77.2058}
XXXXXXXXXXXXXXXX - No nearby places found


Place Searched by   - Indira Park
                          Location: {'lat': 28.5997, 'lng': 77.099}
Nearest Place found - Unnamed Road
                          Location: {'lat': Decimal('28.5997485'), 'lng': Decimal('77.0989718')}


Place Searched by   - PVR Vikaspuri
                   